In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from PIL import Image

In [2]:
train_labels = pd.read_csv("train.csv")
test_labels = pd.read_csv("test.csv")
validation_labels = pd.read_csv("validation.csv")

In [12]:
class CustomImageDataset(Dataset):
    def __init__(self, labels, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(labels)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.img_labels)

    @staticmethod
    def read_image(img_path):
        imagine = np.array(Image.open(img_path))
        imagine = np.ravel(imagine.copy())
        if(imagine.shape == (6400, )):
            imagine = np.concatenate((imagine, imagine, imagine))
        imagine = np.reshape(imagine, (80, 80, 3))
        return np.reshape(imagine, (80, 80, 3))

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]) + ".png"
        #print(img_path)
        image = self.read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [13]:
train_data = CustomImageDataset("train.csv", "train", transform = ToTensor())
test_data = CustomImageDataset("test.csv", "test", transform = ToTensor())
validation_data = CustomImageDataset("validation.csv", "validation", transform = ToTensor())

In [14]:
class ReteaNeuronala(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.first_layer = nn.Linear(80 * 80 * 3, 512)
        self.second_layer = nn.Linear(512, 512)
        self.output_layer = nn.Linear(512, 3)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.first_layer(x))
        x = F.relu(self.second_layer(x))
        x = self.output_layer(x)
        return x        

In [15]:
train_loader = DataLoader(train_data, batch_size = 64)
test_loader = DataLoader(test_data, batch_size = 64)
validation_loader = DataLoader(validation_data, batch_size = 64)

In [16]:
#creem retea si definim algoritmul de optimizare
model = ReteaNeuronala()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)

In [27]:
NUM_EPOCHS = 100
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = model.to(device)
loss_function = nn.CrossEntropyLoss()

model.train(True)
for i in range(NUM_EPOCHS):
    print(f"=== Epoch{i + 1}===")
    for batch, (image_batch, labels_batch) in enumerate(train_loader):
        image_batch = image_batch.to(device)
        labels_batch = labels_batch.to(device)
        #print(image_batch.shape)
        #print(labels_batch.shape)

        pred = model(image_batch)
        #print(pred.shape)
        loss = loss_function(pred, labels_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch%100 == 0:
            loss = loss.item()
            print(f"Batch index{batch}, loss:{loss:>7f}")


cuda
=== Epoch1===
Batch index0, loss:0.977840



KeyboardInterrupt



In [25]:
correct = 0
test_loss = 0
size = len(validation_loader.dataset)
model.to(device)
model.eval()
with torch.no_grad():
    for image_batch, labels_batch in validation_loader:
        image_batch = image_batch.to(device)
        labels_batch = labels_batch.to(device)
        pred = model(image_batch)
        test_loss += loss_function(pred, labels_batch).item()
        correct += (pred.argmax(1) == labels_batch).type(torch.float).sum().item()
correct /= size
test_loss /= size
print(f"Accuracy: {(100*correct):>0.1f}%, Loss: {test_loss:>8f} \n")


Accuracy: 39.5%, Loss: 0.018307 

